In [2]:
using LinearAlgebra

# Kick Stage Preliminary Model

In [11]:
# assuming solid aluminum
ρ_al = 2700

cyl_height = 0.405
cyl_diam = 1.2

cyl_mass = 85
cyl_vol = cyl_mass / ρ_al

cyl_idiam = sqrt(cyl_diam^2 - 4*cyl_mass/(ρ_al * π * cyl_height))

payload_mass = 160
payload_length = cbrt(payload_mass / ρ_al)

0.3898690317617155

In [8]:
ρ_al * payload_length^3

160.00000000000003

In [10]:
# inertia matrices
J_cyl = [
    (cyl_mass/12) * (3 * (cyl_diam^2 + cyl_diam^2) + cyl_height^2) 0 0
    0 (cyl_mass/12) * (3 * (cyl_diam^2 + cyl_diam^2) + cyl_height^2) 0
    0 0 (cyl_mass/2) * (cyl_diam^2 + cyl_idiam^2)
]

J_payload = payload_mass * (payload_length^2 / 6) * I(3)

3×3 Diagonal{Float64, Vector{Float64}}:
 4.05328   ⋅        ⋅ 
  ⋅       4.05328   ⋅ 
  ⋅        ⋅       4.05328

In [ ]:
#I = Ia + Ma(Ra^2 I - RaRa) + Ib + Mb(Rb^2 I - RbRb)

In [17]:
function dyad(v)
    return [
        v[1]*v[1] v[1]*v[2] v[1]*v[3]
        v[2]*v[1] v[2]*v[2] v[2]*v[3]
        v[3]*v[1] v[3]*v[2] v[3]*v[3]
    ]
end

dyad (generic function with 1 method)

In [24]:
# combining objects
# cylinder at origin, cube on top
r_cyl = [0, 0, 0]
r_payload = [0, 0, cyl_height/2 + payload_length/2]

rigidbody_CoM = (payload_mass .* r_payload) ./ (cyl_mass + payload_mass)

Ja = J_cyl + cyl_mass*((r_cyl - rigidbody_CoM).^2 .* I(3) .- dyad(r_cyl))
Jb = J_payload + cyl_mass*((r_payload - rigidbody_CoM).^2 .* I(3) .- dyad(r_payload))

J_rigidbody = Ja + Jb

3×3 Matrix{Float64}:
 66.4151   0.0       0.0
  0.0     66.4151    0.0
  0.0      0.0     116.163

In [40]:
# setting up variables for sizing

# rotate so that spacecraft points along velocity vector
# in LVLH frame (x axis)
Jf = [
    J_rigidbody[3,3] 0 0
    0 J_rigidbody[2,2] 0 
    0 0 J_rigidbody[1,1]
]

# surface area
# outer surface of cylinder + 5 sides of payload
area = π*cyl_diam * cyl_height + 5*payload_length^2

cps = -cyl_height/2 + (cyl_height + payload_length)/2
cpa = cps

0.19493451588085775

In [57]:
# https://sci-hub.mksa.top/10.1038/1831224a0
function kickstage_actuator_sizing(J, orbit_alt, surface_area, r_CoM, cps, cpa; μ=3.986e14, lv_deviation=deg2rad(1), q=0.8, i=0, D=1, Cd=2.25, ρ=1e-10, v=7784.338495550994)
    r_orbit = 6378e3 + orbit_alt*1e3
    T_gravity_gradient = (3*μ / (2 * r_orbit^3)) * abs(J[1,1] - J[2,2]) * sin(deg2rad(2 * lv_deviation))
    
    r_cps = [cps, 0, 0]
    F_solar = (1367 / 3e8) * surface_area * (1 + q) * cos(i)
    T_solar_pressure = F_solar * (r_cps[1] - r_CoM[1])
    
    T_magnetic = D * 2 * 7.96e15 / r_orbit^3
    
    r_cpa = [cpa, 0, 0]
    F_aerodynamic = 0.5 * ρ * Cd * surface_area * v^2
    T_aerodynamic = F_aerodynamic * (r_cpa[1] - r_CoM[1])
    
    return T_gravity_gradient, T_solar_pressure, T_magnetic, T_aerodynamic
end

kickstage_actuator_sizing (generic function with 1 method)

In [58]:
kickstage_actuator_sizing(Jf, 200, area, rigidbody_CoM, cps, cpa)

(6.36658294189817e-8, 3.65626214903134e-6, 5.593214256632894e-5, 0.003038879708349613)

In [59]:
1.25*0.003039

0.00379875

# Actuator Sizing

In [5]:
design_torque = 0.003799

Jf = [
    66.4151 0.0 0.0
    0.0 66.4151 0.0
    0.0 0.0 116.163
]

3×3 Matrix{Float64}:
 66.4151   0.0       0.0
  0.0     66.4151    0.0
  0.0      0.0     116.163

In [6]:
# slew torque for reaction wheels
inertia = sqrt(Jf[1,1]^2 + Jf[2,2]^2 + Jf[3,3]^2)
slew = deg2rad(30)
time = 600

slew_torque = 4 * slew * inertia / time^2

0.0008690846634383133

In [7]:
# momentum storage in reaction wheel
period = (2*π / sqrt(398600)) * (6378 + 200)^(3/2)
momentum = design_torque * (period / 4) * 0.707

3.5651740842842705

In [13]:
# thruster force for external disturbances
moment_arm = cyl_diam / 2
disturbance_thrust = design_torque / moment_arm

0.006331666666666667

In [14]:
# thruster force for momentum dumping
time = 1
momentum_thrust = momentum / (moment_arm * time)

5.941956807140451